# Fashion MNIST Codeology

No GPU required! Run this on your potato.

The basic 7 steps for building models in general are listed so:
 1. Load Dataset
 2. Make Dataset Iterable
 3. Create Model Class
 4. Instantiate Model Class
 5. Instantiate Loss Class
 6. Instantiate Optimizer Class
 7. Train Model

Download data from here! https://drive.google.com/drive/folders/1NdPvylq73-kSRSKCiJBTrzhut-k6pAkv?usp=sharing

### Run the below cells to get your data processed and loaded

In [1]:

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader

In [2]:
'''
STEP 1: LOAD DATASET
'''
test_df = pd.read_csv('fashionmnist/fashion-mnist_test.csv')
test_df_labels = test_df['label']
test_pixels_df = test_df.drop('label', axis=1)

'''
If you're curious about how I did this see the below cells. If not just skip to STEP 1.5

Pandas is a library for dataprocessing. You might run into dask.DataFrame at some point if you continue with ML.
dask.DataFrame is built ontop of Pandas with the purpose of concurrency and parallelized computing...basically when
working with datasets so large that you require multiple machines to handle it. This is part of the data pipeline!
'''

# This reads the csv file into a pandas dataframe
train_df = pd.read_csv('fashionmnist/fashion-mnist_train.csv')
train_df.head()

label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      2       0       0       0       0       0       0       0       0   
1      9       0       0       0       0       0       0       0       0   
2      6       0       0       0       0       0       0       0       5   
3      0       0       0       0       1       2       0       0       0   
4      3       0       0       0       0       0       0       0       0   

   pixel9    ...     pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0        30        43   
3       0    ...            3         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  pixel784  
0         0         0         0         0         0  
1         0         0         0         0         0  
2         0         0         0         0         0  
3         1         0         0         0         0  
4         0         0         0         0         0  

[5 rows x 785 columns]

In [3]:
# We create a new dataframe without the 'label' column here so we only get the pixel data
# The original dataframe train_df is unmodified
train_pixels_df = train_df.drop('label', axis=1)
train_pixels_df.head()

pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       5       0   
3       0       0       0       1       2       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel10    ...     pixel775  pixel776  pixel777  pixel778  pixel779  \
0        0    ...            0         0         0         0         0   
1        0    ...            0         0         0         0         0   
2        0    ...            0         0         0        30        43   
3        0    ...            3         0         0         0         0   
4        0    ...            0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  pixel784  
0         0         0         0         0         0  
1         0         0         0         0         0  
2         0         0         0         0         0  
3         1         0         0         0         0  
4         0         0         0         0         0  

[5 rows x 784 columns]

In [4]:
# Now we grab only the labels. Keep in mind that we do not change the order of either the pixel values nor the labels
# so that they stay consistent
train_df_labels = train_df['label']
train_df_labels.values

array([2, 9, 6, ..., 8, 8, 7], dtype=int64)

In [12]:
'''
STEP 1.5: defining and instantiating Dataset subclass 
'''

'''
This is our custom Dataset class. Remember from 1st meeting that we need this to pipeline our data into training our model.

The pipeline is important!!! At larger scale, machine learning can get bottlenecked at disk reads (in image classification for example)
so understanding the various stages is important. We don't have to worry about that kind of stuff now since we're just creating small
project models as opposed to complex production models.

NOTE: this is not the only way to create a dataset. An alternative is to simply pass in a dataframe that contains both pixel and label data.
Then we can index the label and pixel data inside of __getitem__ as opposed to separating labels and pixel data before hand like I did.
'''
class FashionDataset(Dataset):
    def __init__(self, dataframe, labels):
        self.labels = torch.LongTensor(labels)
        self.df = dataframe
        
    def __getitem__(self, index):
        # I'm using .loc to access the row of the dataframe by index
        a = self.df.loc[index]
        a = (a - np.mean(a))/np.std(a)
        img = torch.Tensor(a)
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.labels)
    
'''
This class is for providing image data as (1, 28, 28) tensor as opposed to a (784) tensor. You
use these for conv2d layers which are powerful for image recognition!
'''
class Fashion2DDataset(Dataset):
    def __init__(self, dataframe, labels):
        self.labels = torch.LongTensor(labels)
        self.df = dataframe
        
    def __getitem__(self, index):
        # I'm using .loc to access the row of the dataframe by index
        a = self.df.loc[index]
        a = (a - np.mean(a))/np.std(a)
        a = np.split(a, 28)
        a = np.array([a])
        img = torch.Tensor(a)
        
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.labels)

In [13]:
'''
STEP 2: MAKING DATASET ITERABLE
'''
# train_dataset = Fashion2DDataset(train_pixels_df, train_df_labels.values)
# test_dataset = Fashion2DDataset(test_pixels_df, test_df_labels.values)

train_dataset = FashionDataset(train_pixels_df, train_df_labels.values)
test_dataset = FashionDataset(test_pixels_df, test_df_labels.values)

'''
Batch_size will determine how many data samples to go through before 
updating the weights of our model with SGD (stochastic gradient descent)

Currently at 100 but feel free to change this to whatever you want. You can consider
batch size a hyper parameter!
'''
batch_size = 100

# shuffle is true so that we train our data on all labels simultaneously. The data is already shuffled in 
# this case(You can verify this by looking through the training labels by running train_labels in its own cell)
# If this wasn't the case, and we had shuffle=False, we might end up training the model on label = 0 and 
# then ending with label = 9. This would cause the model to 'forget' what label = 0 looked like
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

# shuffle=False because theres no reason to do so with testing
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [14]:
'''
STEP 3: CREATE MODEL CLASS
'''
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
class ConvolutionalNeuralNetModel(nn.Module):
    def __init__(self, output_dim):
        super(ConvolutionalNeuralNetModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=4, stride=2, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.fc = nn.Linear(1024, output_dim)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
# model = ConvolutionalNeuralNetModel(output_dim)


In [10]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

In [11]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
"""
Most of the time I use SGD. Feel free to use another optimizer if you wish.
What hyperparameters would you use/set here?
"""
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [32]:
'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
losses = []
accuracies = []
for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader):
        # Load images with gradient accumulation capabilities
        images = images.view(-1, 28*28).requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 60 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images with gradient accumulation capabilities
                images = images.view(-1, 28*28).requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
            
            accuracies.append(accuracy)
            losses.append(loss.item())
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 60. Loss: 0.3144973814487457. Accuracy: 87
Iteration: 120. Loss: 0.2591107189655304. Accuracy: 86
Iteration: 180. Loss: 0.25049054622650146. Accuracy: 87
Iteration: 240. Loss: 0.30468255281448364. Accuracy: 86
Iteration: 300. Loss: 0.22775261104106903. Accuracy: 88
Iteration: 360. Loss: 0.35631152987480164. Accuracy: 87
Iteration: 420. Loss: 0.26784151792526245. Accuracy: 86
Iteration: 480. Loss: 0.43664658069610596. Accuracy: 85
Iteration: 540. Loss: 0.36955925822257996. Accuracy: 88
Iteration: 600. Loss: 0.323708176612854. Accuracy: 87


In [10]:
iterations = np.arange(60, 601, 60)
iterations
lossesnp = np.array(losses)*100

In [11]:
import matplotlib.pyplot as plt

In [9]:
plt.plot(iterations, lossesnp)
plt.plot(iterations, accuracies)
plt.xticks(np.arange(0, 601, 60))
plt.yticks(np.arange(0,101,10))
plt.ylabel('Percent')
plt.xlabel('Iteration (or batch)')
plt.title('Accuracy and Loss over time')
plt.show()

C:\Users\HuyenLe\Anaconda3\lib\site-packages\matplotlib\axes\_axes.py:531: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [25]:
def retrieve(index):
    img = test_pixels_df.loc[index]
    img2d = np.split(img, 28)
    plt.imshow(img2d, cmap='gray')
    plt.show()
    return img

def predict(arr, labels, model):
    img = torch.Tensor(arr)
    prediction = model(img)
    value, index = prediction.max(0)
    return labels[index.item()]

LABELS= {
    0:'T-shirt/top',
    1:'Trouser',
    2:'Pullover',
    3:'Dress',
    4:'Coat',
    5:'Sandal',
    6:'Shirt',
    7:'Sneaker',
    8 :'Bag',
    9 :'Ankle boot' 
}

In [28]:
arr = retrieve(100)
predict(arr, LABELS, model)

'T-shirt/top'